# **VISCOSITY SOLUTIONS THROUGH PENALTY METHODS**

## **Code**

### Firedrake

In [ ]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2025-12-15 18:24:05--  https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4767 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0.001s  

2025-12-15 18:24:05 (8.66 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4767/4767]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#############################################################

### Other

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

### Burgers

In [ ]:
def burgers(h=2**-10, degree=0, nu=0, timestep=2**-5, end_time=1.0, sigma=2**-2):
    mesh = UnitIntervalMesh(round(1/h))
    n = FacetNormal(mesh)
    x, = SpatialCoordinate(mesh)

    V = FunctionSpace(mesh, "DG", degree)
    u_ = Function(V, name="VelocityOld")
    u = Function(V, name="Velocity")
    v = TestFunction(V)

    ic = project((
        conditional(le(abs(x - 0.25), 0.125), 1, 0)
      + conditional(le(abs(x - 0.75), 0.125), - 0.5, 0)
    ), V)

    u_.assign(ic)
    u.assign(ic)

    jump = lambda u: -2 * avg(u * n[0])

    F = (
        inner((u - u_)/timestep, v) * dx
        + 2/3 * inner(avg(u) * jump(u), avg(v)) * dS
        + 1/3 * inner(avg(u * v), jump(u)) * dS
        + sigma * inner(jump(u), jump(v)) * dS
    )
    if degree != 0:
        F += inner(u * u.dx(0), v) * dx
    if nu != 0:
        F += nu * inner(u.dx(0), v.dx(0)) * dx
    if degree != 0 and nu != 0:
        F += (
            inner(avg(u.dx(0)), jump(v))
          + inner(jump(u), avg(v.dx(0)))
        ) * dS

    sp = {
        # "snes_monitor": None,
        # "snes_converged_reason": None,
        "snes_max_it": 100,
    }

    fig, ax = plt.subplots(figsize=(10, 6))
    state = {'t': 0.0}

    def update(frame):
        if frame > 0:
            state['t'] += timestep
            print(GREEN % f"Solving for time t = {state['t']:.4f}:")
            solve(F == 0, u, solver_parameters=sp)
            u_.assign(u)

        ax.clear()
        plot(u, axes=ax, linewidth=3)
        ax.set_title(f"Burgers equation (t = {state['t']:.2f})")
        ax.set_xlabel("x")
        ax.set_ylabel("u")
        ax.set_ylim(-1, 1.5)
        ax.grid(True)

    num_frames = int(end_time / timestep) + 1
    anim = FuncAnimation(fig, update, frames=num_frames, interval=100)
    plt.close()
    return HTML(anim.to_jshtml())

---

## **The idea**

### Example 1: *Burgers*

### Example 2: *Allen–Cahn*

### Example 3: *Navier–Stokes(/Euler)*